In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

relative_path = '../../sourcedata/'

# 读取题目信息 title = 题目
title_info = pd.read_csv(relative_path + 'Data_TitleInfo.csv')

# 读取学生信息 student = 学生
student_info = pd.read_csv(relative_path + 'Data_StudentInfo.csv')

In [6]:
def classname_validate(classname, submit_record):
    # 检查submit_record中每一行数据的class是否与classname一致
    not_match_rows = submit_record[submit_record['class'] != classname]
    if not_match_rows.shape[0] > 0:
        print('classname_validate length: ', len(not_match_rows))
    
    # 将对应数据修改为classname
    for index, row in submit_record.iterrows():
        submit_record.loc[index, 'class'] = classname




# 检查分数与状态是否匹配，分数是否合法，不满足的直接删除
def score_state_validate(submit_record):
    submit_record.loc[:, 'full_score'] = pd.to_numeric(submit_record.loc[:, 'full_score'],errors='coerce')
    submit_record.loc[:, 'score'] = pd.to_numeric(submit_record.loc[:, 'score'],errors='coerce')
    # 检查submit_record中每一行数据的score和state是否符合要求
    # 其中，state=Absolutely_Correct时，score=full_score; state=Partially_Correct时，score=(0,full_score),不取到两端的值; 其他state时，score=0

    not_match_rows = []
    for index, row in submit_record.iterrows():
        if row['state'] == 'Absolutely_Correct' and row['score'] != row['full_score']:
            not_match_rows.append(index)
        elif row['state'] == 'Partially_Correct' and (row['score'] <= 0 or row['score'] >= row['full_score']):
            not_match_rows.append(index)
        elif row['state'] != 'Absolutely_Correct' and row['state'] != 'Partially_Correct' and row['score'] != 0:
            not_match_rows.append(index)
    if len(not_match_rows) > 0:
        print('score_state_validate: 即将删除以下索引代表的行数据：\n', not_match_rows)
        # 由于无法判定score和state的关系，所以全部删除
        
        not_match_rows = list(set(not_match_rows))  
        submit_record.drop(not_match_rows, inplace=True)
        print('删除后的数据量：', submit_record.shape[0])







# 检查提交时间是否合法（to_numeric全部有效）
# 检查时间，内存消耗是否合法，不合法的设置为nan --->保存为空
# 缺失值不做处理 --->保存为空
# 答题状态为所指定的状态，不满足的直接删除
# method为Method_开头的字符串，不满足的直接删除    
def blank_abnormal_check(submit_record):
    
    abnormal_rows = []
    null_rows = []
    err_time = []
    # submit_record = submit_record.fillna(0)
    submit_record.loc[:, 'timeconsume'] = pd.to_numeric(submit_record.loc[:, 'timeconsume'],errors='coerce')
    submit_record.loc[:, 'memory'] = pd.to_numeric(submit_record.loc[:, 'memory'],errors='coerce')
    submit_record.loc[:, 'time'] = pd.to_numeric(submit_record.loc[:, 'time'],errors='raise') # 做时间有效性检查
    
    for index, row in submit_record.iterrows():
        # 检查timeconsume列中是负值，非数字值行
        # if not isinstance(row['timeconsume'], (int, float)) or row['timeconsume'] < 0:
        #     abnormal_rows.append(index)

        # 检查所有存在空值的行
        if row.isnull().sum() > 0: # isnull().sum()返回每行的空值数量
            null_rows.append(index)

        # 检查memory列中是负值，非数字值行
        # if not isinstance(row['memory'], (int, float)) or row['memory'] < 0:
        #     abnormal_rows.append(index)

        # 检查state列中存在非Absolutely_Correct, Partially_Correct, Absolutely_Error, Error1, Error2, Error3, Error4, Error5, Error6, Error7, Error8, Error9的行
        if row['state'] not in ['Absolutely_Correct', 'Partially_Correct', 'Absolutely_Error', 'Error1', 'Error2', 'Error3', 'Error4', 'Error5', 'Error6', 'Error7', 'Error8', 'Error9']:
            abnormal_rows.append(index)
        
        # 检查method的合法性，必须是Method_开始的字符串
        if not row['method'].startswith('Method_'):
            abnormal_rows.append(index)

        # 检查time是否是合理的时间，例如正常的数据为1704209872这种形式

    # 三个数组去重
    abnormal_rows = list(set(abnormal_rows))
    null_rows = list(set(null_rows))
    err_time = list(set(err_time))    
    
    if len(null_rows) > 0:
        print('blank_check length', len(null_rows))

    if len(abnormal_rows) > 0:
        print('blank_abnormal_check: \n', abnormal_rows)
        submit_record.drop(abnormal_rows, inplace=True)
        print('删除后的数据量：', submit_record.shape[0])
    
    if len(err_time) > 0:
        print('err_time length', len(err_time), err_time)

        
    

In [7]:
# 读取Data_SubmitRecord文件夹中的所有csv文件
for file in os.listdir(relative_path + 'Merged_Data_SubmitRecord'):
    print(file + ' check start!')
    submit_record =  pd.read_csv(relative_path + 'Merged_Data_SubmitRecord/' + file)
    # 从file(SubmitRecord-Class1)提取出班级
    class_name = file.split('-')[1].split('.')[0]

    print('class_name:', class_name)
    classname_validate(class_name, submit_record)
    score_state_validate(submit_record)
    blank_abnormal_check(submit_record)

    # submit_record.to_csv(relative_path + 'Merged_Data_SubmitRecord/' + file, index=False) # 保存文件

    print(file + ' check end and saved!\n')
    # break


SubmitRecord-Class1.csv check start!
class_name: Class1
blank_check length 332
SubmitRecord-Class1.csv check end and saved!

SubmitRecord-Class10.csv check start!
class_name: Class10
blank_check length 119
SubmitRecord-Class10.csv check end and saved!

SubmitRecord-Class11.csv check start!
class_name: Class11
blank_check length 128
SubmitRecord-Class11.csv check end and saved!

SubmitRecord-Class12.csv check start!
class_name: Class12
blank_check length 165
SubmitRecord-Class12.csv check end and saved!

SubmitRecord-Class13.csv check start!
class_name: Class13
blank_check length 197
SubmitRecord-Class13.csv check end and saved!

SubmitRecord-Class14.csv check start!
class_name: Class14
blank_check length 126
SubmitRecord-Class14.csv check end and saved!

SubmitRecord-Class15.csv check start!
class_name: Class15
blank_check length 250
SubmitRecord-Class15.csv check end and saved!

SubmitRecord-Class2.csv check start!
class_name: Class2
blank_check length 254
SubmitRecord-Class2.csv chec